# Google Colaboratory の準備

本ファイルは、`notebook/1-4_inference.ipynb`をGoogle Colaboratoryで実行するためのノートブックです。  
あらかじめ、`notebook/colab/1-0_prepare_colab.ipynb`の手順を実行し、Googleドライブにリファレンス環境（ソフトウェア）のリポジトリとデータセット・学習済みモデルの準備が完了していることを確認してください。

## GPUの指定

Google Colaboratoryのメニューバーの「ランタイム」から「ランタイムのタイプを変更」を選択し、
ハードウェア アクセラレータを**GPU**に指定した上で、本ファイルを実行してください。

## ライブラリのインストール

In [ ]:
!pip install -q torch==1.7.1+cu110 torchvision==0.8.2+cu110 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -q nuscenes-devkit

!pip install -q fire tensorboardX protobuf opencv-python
!pip install -q numba scikit-image scipy pillow

     |███████████████████████         | 834.1 MB 1.2 MB/s eta 0:04:27tcmalloc: large alloc 1147494400 bytes == 0x39730000 @  0x7f3d7d01d615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████████████████▏  | 1055.7 MB 1.1 MB/s eta 0:01:29tcmalloc: large alloc 1434370048 bytes == 0x7dd86000 @  0x7f3d7d01d615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████████████████| 1156.7 MB 95.8 MB/s eta 0:00:01tcmalloc: large alloc 1445945344 bytes == 0xd3572000 @  0x7f3d7d01d615 0x592b76 0x4df71e 0x59afff 0x515655 0

In [ ]:
# spconv v1.2.1 のインストール

!git clone --depth 1 --recursive  -b v1.2.1 https://github.com/traveller59/spconv.git
%cd spconv
!SPCONV_FORCE_BUILD_CUDA=1 python setup.py bdist_wheel
!pip install ./dist/spconv*

Cloning into 'spconv'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 118 (delta 16), reused 32 (delta 1), pack-reused 0
Receiving objects: 100% (118/118), 234.67 KiB | 8.09 MiB/s, done.
Resolving deltas: 100% (16/16), done.
Submodule 'third_party/cutlass' (https://github.com/NVIDIA/cutlass) registered for path 'third_party/cutlass'
Submodule 'third_party/mp11' (https://github.com/boostorg/mp11) registered for path 'third_party/mp11'
Submodule 'third_party/pybind11' (https://github.com/pybind/pybind11.git) registered for path 'third_party/pybind11'
Cloning into '/content/spconv/third_party/cutlass'...
remote: Enumerating objects: 16512, done.        
remote: Counting objects: 100% (567/567), done.        
remote: Compressing objects: 100% (297/297), done.        
remote: Total 16512 (delta 257), reused 529 (delta 249), pack-reused 15945        
Receiving objects: 100% (16512/1651

## Googleドライブをマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# WORK_DIRのパスは自身の作業ディレクトリに変更してください。

WORK_DIR = "/content/drive/MyDrive/Colab Notebooks/work/"
%cd $WORK_DIR

# パスを合わせるため、リポジトリのnotebookディレクトリに移動
%cd ./6th-ai-reference2/notebook

/content/drive/MyDrive/Colab Notebooks/work
/content/drive/MyDrive/Colab Notebooks/work/6th-ai-reference2/notebook


以下より、`notebook/1-4_inference.ipynb`と同じ内容です。

# 概要

本ファイルは、学習済みモデルを使用して、3D物体検出モデルの推論・評価を実行します。 

> 本ファイルで実行するスクリプトはsecond.pytorchをベースとしています。

## Output
本ファイルを実行することで、`results`ディレクトリに、推論結果(result.pkl)を出力します。

```
└ second.pytorc
     └ checkpoints
          └ {model_dir}
                ├ results    <- 本ファイルにて生成されます
                   └ step_xxxx    
                       └─ result.pkl
```

# 準備

あらかじめ、 `data`ディレクトリに、`3d_labels_painted`ディレクトリと、`second.pytorch/checkpoints/{model_dir}` ディレクトリに学習済みモデルがあることを確認してください。

> 本ファイルのデフォルトでは、公開の学習済みモデル`second.pytorch/checkpoints/pointpainting`を設定しています。


```
└ data
    ├ 3d_labels
    └ 3d_labels_painted
        ├ maps
        ├ samples
            ├ CAM_FRONT
            └ LIDAR_TOP
        ├ v1.0-trainval
        ├ gt_database        <- 1-2_create_dataset_metadata_colab.ipynb にて生成
        ├ infos_train.pkl    <- 1-2_create_dataset_metadata_colab.ipynb にて生成
        ├ infos_val.pkl      <- 1-2_create_dataset_metadata_colab.ipynb にて生成
        └ kitti_dbinfos_train.pkl  <- 1-2_create_dataset_metadata_colab.ipynb にて生成
 
├second.pytorch
     └─ checkpoints
          └ {model_dir}
                ├── checkpoints.json
                ├── {model-step}.tckpt <- 学習済みモデル
```

# 推論・評価

検証データ(val)に対して推論と評価を実施します。  

> 学習データと検証データはnuScenesデータセットで定義されています。  
> 検証データはscene-0094とscene-0109 の2シーンで、
> 学習データはそれ以外の35シーンです。

In [ ]:
# パスの設定

config_path = "../second.pytorch/second/configs/nuscenes/pointpainting.config"

model_dir = "../second.pytorch/checkpoints/pointpainting"

In [ ]:
!python ../second.pytorch/second/pytorch/train.py evaluate \
        --config_path=$config_path \
        --model_dir=$model_dir --measure_time=True --batch_size=4

[  41 1984  992]
Restoring parameters from /content/drive/MyDrive/Colab Notebooks/work/6th-ai-reference2/second.pytorch/checkpoints/pointpainting/voxelnet-24000.tckpt
feature_map_size [1, 248, 124]
Generate output labels...
[100.0%][===================>][1.09it/s][00:43>00:00]   
generate label finished(4.32/s). start eval:
avg example to torch time: 6.228 ms
avg prep time: 784.751 ms
avg voxel_feature_extractor time = 0.368 ms
avg middle forward time = 74.617 ms
avg rpn forward time = 30.031 ms
avg predict time = 23.925 ms
Evaluation nusc
Nusc v1.0-trainval Evaluation
car Nusc dist AP@0.5, 1.0, 2.0, 4.0 and TP errors
4.76, 6.25, 7.27, 7.33
trans_err, scale_err, orient_err, vel_err, attr_err: 0.2853, 0.2348, 2.4496, 1.0000, 0.9783
pedestrian Nusc dist AP@0.5, 1.0, 2.0, 4.0 and TP errors
0.00, 0.00, 0.00, 0.00
trans_err, scale_err, orient_err, vel_err, attr_err: 1.0000, 1.0000, 1.0000, 1.0000, 1.0000



## （オプション）推論結果の確認

推論データは pklファイルに保存されています。pklファイルを読み込みことで推論データを確認することができます。

In [ ]:
import pickle

result_pkl = "../second.pytorch/checkpoints/pointpainting/eval_results/step_24000/result.pkl"

with open(result_pkl, 'rb') as f:
    result = pickle.load(f)

print("result size: ", len(result))
print(result[0]["box3d_lidar"])  # (n, 7)  [x, y, z, l, h, w, rz]
print(result[0]["scores"])  # (n)
print(result[0]["label_preds"])  # (n)  0: car, 1:pedestrian

result size:  187
tensor([[ 13.5258,  -3.0029,  -0.9799,   1.8181,   4.2832,   1.7754,   1.5867],
        [ 20.1477,   0.6291,  -1.1004,   1.8185,   4.2175,   1.7506,   1.5452],
        [ 11.2833,  -6.6473,  -1.3338,   1.6925,   3.8949,   1.6504,  -1.4243],
        [ 29.8656, -15.1257,  -1.7504,   1.6665,   3.7216,   1.6058,  -1.3598],
        [ 41.2429,   8.1982,  -0.7760,   0.5630,   0.6865,   1.6484,   1.6287],
        [ 20.5917,   8.8068,  -0.5630,   0.5593,   0.6854,   1.6449,  -1.9882]],
       device='cuda:0')
tensor([0.7925, 0.6245, 0.5771, 0.3633, 0.3603, 0.3062], device='cuda:0')
tensor([0, 0, 0, 0, 1, 1], device='cuda:0')


以上